In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras import models
from keras.layers import Dense, Conv1D, Reshape, Flatten, Lambda
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
import keras

Using TensorFlow backend.
/home/inswave/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/inswave/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/inswave/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/inswave/anaconda3/lib/python3.7/site-packages/t

In [2]:
class Data:
    def __init__(self, mu, sigma, ni_D):
        self.mu = mu
        self.sigma = sigma
        self.ni_D = ni_D
        #self.real_sample = lambda n_batch: np.random.normal(mu, sigma, (n_batch, ni_D))
        #self.in_sample = lambda n_batch: np.random.rand(n_batch, ni_D)

    def real_sample(self, n_batch):
        return np.random.normal(self.mu, self.sigma, (n_batch, self.ni_D))

    def in_sample(self, n_batch):
        return np.random.rand(n_batch, self.ni_D)


In [3]:
class Machine:
    def __init__(self, n_batch=10, ni_D=100):
        data_mean = 4
        data_stddev = 1.25

        self.n_iter_D = 5
        self.n_iter_G = 5

        self.data = Data(data_mean, data_stddev, ni_D)
        self.gan = GAN(ni_D=ni_D, nh_D=50, nh_G=50)
        self.n_batch = n_batch

    def train_D(self):
        gan = self.gan
        n_batch = self.n_batch
        data = self.data
        Real = data.real_sample(n_batch)
        Z = data.in_sample(n_batch)

        Gen = gan.G.predict(Z)
        gan.D.trainable = True
        #print('train_D')
        gan.D_train_on_batch(Real, Gen)

    def train_GD(self):
        gan = self.gan
        n_batch = self.n_batch
        data = self.data
        Z = data.in_sample(n_batch)

        gan.D.trainable = False
        #print('train_GD')
        gan.GD_train_on_batch(Z)

    def train_each(self, epoch):
        #print('train_each - 0:', epoch)
        for it in range(self.n_iter_D):
            #print('train_each - 1:', epoch)
            self.train_D()
        for it in range(self.n_iter_G):
            #print('train_each - 2:', epoch)
            self.train_GD()

    def train(self, epochs):
        for epoch in range(epochs):
            self.train_each(epoch)

    def test(self, n_test):
        gan = self.gan
        data = self.data
        Z = data.in_sample(n_test)
        Gen = gan.G.predict(Z)
        return Gen, Z

    def show_hist(self, Real, Gen, Z):
        plt.hist(Real.reshape(-1), histtype='step', label='Real')
        plt.hist(Gen.reshape(-1), histtype='step', label='Generated')
        plt.hist(Z.reshape(-1), histtype='step', label='Input')
        plt.legend(loc=0)

    def test_and_show(self, n_test):
        data = self.data
        Gen, Z = self.test(n_test)
        Real = data.real_sample(n_test)
        self.show_hist(Real, Gen, Z)
        self.print_stat(Real, Gen)

    def run_epochs(self, epochs, n_test):
        self.train(epochs)
        self.test_and_show(n_test)

    def run(self, n_repeat=200, n_show=200, n_test=100):
        for ii in range(n_repeat):
            print('Stage', ii, '(Epoch: {})'.format(ii * n_show))
            self.run_epochs(n_show, n_test)
            plt.show()

    def print_stat(self, Real, Gen):
        def stat(d):
            return (np.mean(d), np.std(d))
        print('Mean and Std of Real:', stat(Real))
        print('Mean and Std of Gen:', stat(Gen))


In [4]:
def add_decorate(x):
    m = K.mean(x, axis=-1, keepdims=True)
    d = K.square(x - m)
    return K.concatenate([x,d], axis=-1)

def add_decorate_shape(input_shape):
    shape = list(input_shape)
    assert len(input_shape) == 2
    shape[1] *= 2
    return tuple(shape)

lr = 2e-4
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999)

def model_compile(model):
    return model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [5]:
class GAN:
    def __init__(self, ni_D, nh_D, nh_G):
        self.ni_D = ni_D
        self.nh_D = nh_D
        self.nh_G = nh_G
        
        self.D = self.gen_D()
        self.G = self.gen_G()
        self.GD = self.make_GD()
    
    def gen_D(self):
        ni_D = self.ni_D
        nh_D = self.nh_D
        
        D = models.Sequential()
        D.add(Lambda(add_decorate, output_shape=add_decorate_shape, input_shape=(ni_D,)))
        
        D.add(Dense(nh_D, activation='relu'))
        D.add(Dense(nh_D, activation='relu'))
        D.add(Dense(1, activation='sigmoid'))
        
        model_compile(D)

        print('D.summary()')
        D.summary()
        return D
    
    def gen_G(self):
        ni_D = self.ni_D
        nh_G = self.nh_G
        
        G = models.Sequential()
        G.add(Reshape((ni_D, 1), input_shape=(ni_D,)))
        G.add(Conv1D(nh_G, 1, activation='relu'))
        G.add(Conv1D(nh_G, 1, activation='sigmoid'))
        G.add(Conv1D(1, 1))
        G.add(Flatten())
        
        model_compile(G)
        
        print('G.summary()')
        G.summary()
        return G
    
    def make_GD(self):
        G, D = self.G, self.D
        GD = models.Sequential()
        GD.add(G)
        GD.add(D)
        D.trainable = False
        model_compile(GD)
        print('GD.summary()')
        GD.summary()
        D.trainable = True
        return GD
    
    def D_train_on_batch(self, Real, Gen):
        D = self.D
        X = np.concatenate([Real, Gen], axis=0)
        y = np.array([1]*Real.shape[0] + [0]*Gen.shape[0])
        
        #print('X.shape',X.shape)
        #print('y.shape',y.shape)
        #print(X)        
        #print(y)
        D.train_on_batch(X, y)
#        D.fit(X,y)
        
    def GD_train_on_batch(self, Z):
        GD = self.GD
        y = np.array([1]*Z.shape[0])
        #print('Z.shape',Z.shape)
        #print('y.shape',y.shape)
        #print(Z)        
        #print(y)
        
        
        GD.train_on_batch(Z, y)
#        GD.fit(Z, y)

In [ ]:
machine = Machine(n_batch=1, ni_D=1000)
machine.run(n_repeat=400, n_show=100, n_test=100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
D.summary()
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 2000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                100050    
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 102,651
Trainable params: 102,651
Non-trainable params: 0
_________________________________________________________________
G.summary()
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shap

In [17]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

1.15.2
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6057242958502736950
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 589915257607230299
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15914857147868314031
physical_device_desc: "device: XLA_GPU device"
]
